# Protypical Networks

Keras implementation

In [1]:
%pylab inline

import os
import logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.INFO, format='%(message)s')
# logging.basicConfig(level=logging.INFO)
Logger = logging.getLogger('protonet')
Logger.setLevel(logging.INFO)

# import skimage, skimage.io, skimage.transform

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf

from tensorflow import keras, nn

2021-08-19 23:08:25.887156: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [5]:
import fsl

## Load data

In [6]:
datadir = os.path.expanduser('~/Database/Machine_Learning')
workdir = os.getcwd()

outdir = f'{workdir}/outputs'

try:
    os.makedirs(outdir)
except Exception as msg:
    print(msg)
    pass

[Errno 17] File exists: '/home/han/Projects/Few Shot Learning/Notebooks/Prototype_Networks/outputs'


### Data loader

In [7]:
# ## On the original dataset
# indir = f'{datadir}/Omniglot/omniglot.original/python/'

# foo1 = nd.from_folder(f'{indir}/images_background')
# foo2 = nd.from_folder(f'{indir}/images_evaluation')

# data_dict = {'images_background': foo1, 'images_evaluation': foo2}
# # print(len(data_dict.keys()))

# ms = fsl.utils.data.Meta_Sampler(data_dict, 0.75, lvl_split=1, lvl_task=2, n_way=20, k_shot=5)

# print(len(ms.trainval_dict.keys()))

# print(len(ms.test_dict.keys()))

In [8]:
## On the fused dataset
indir = f'{datadir}/Omniglot/omniglot.fused/data/'
train_dict, test_dict = fsl.utils.sampler.split_dict_from_folder(indir, 0.75, 0)

# data_dict = fsl.utils.nested_dict.from_folder(indir)
# train_dict, test_dict = fsl.utils.nested_dict.random_split_level(data_dict, 0.75, 0)

# train_sampler = fsl.utils.sampler.Meta_Sampler(train_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)
# test_sampler = fsl.utils.sampler.Meta_Sampler(test_dict, indir, lvl_task=1, n_way=10, k_shot=5, q_shot=None)
# test_sampler = train_sampler

In [11]:
kwargs_imread = dict(invert=True, dim=(25,25))

func = lambda x: fsl.utils.data.imread(x, **kwargs_imread)
train_sampler = fsl.utils.sampler.SequentialSampler(train_dict, indir, lvl_task=1, \
                                               n_way=10, k_shot=5, q_shot=None, local=False, func=func, split=True)
test_sampler = fsl.utils.sampler.SequentialSampler(test_dict, indir, lvl_task=1, \
                                               n_way=10, k_shot=5, q_shot=None, local=False, func=func, split=True)
# train_sampler = fsl.utils.sampler.Meta_Sampler(train_dict, indir, lvl_task=1)


## Model construction

In [15]:
# model = fsl.models.tf.ProtoNet((*kwargs_imread['dim'],1))
model = fsl.models.tf.ProtoNet.get_model((*kwargs_imread['dim'],1), train_sampler.n_way, train_sampler.k_shot)
# model = fsl.models.tf.get_model(kwargs_imread['dim'])

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseTopKCategoricalAccuracy(k=1)],
              optimizer=keras.optimizers.Adam(learning_rate=5e-4))
# model.summary()

In [ ]:
res = model.fit(train_sampler, validation_data=test_sampler, epochs=100, use_multiprocessing=False)

Epoch 1/100
120/120 [==============================] - 14s 110ms/step - loss: 4.0277 - sparse_top_k_categorical_accuracy: 0.0250 - val_loss: 2.4908 - val_sparse_top_k_categorical_accuracy: 0.0429
Epoch 2/100
120/120 [==============================] - 12s 98ms/step - loss: 2.4407 - sparse_top_k_categorical_accuracy: 0.0466 - val_loss: 2.4076 - val_sparse_top_k_categorical_accuracy: 0.0476
Epoch 3/100
120/120 [==============================] - 13s 108ms/step - loss: 2.3811 - sparse_top_k_categorical_accuracy: 0.0534 - val_loss: 2.3746 - val_sparse_top_k_categorical_accuracy: 0.0509
Epoch 4/100
120/120 [==============================] - 13s 107ms/step - loss: 2.3541 - sparse_top_k_categorical_accuracy: 0.0628 - val_loss: 2.3567 - val_sparse_top_k_categorical_accuracy: 0.0558
Epoch 5/100
120/120 [==============================] - 13s 109ms/step - loss: 2.3404 - sparse_top_k_categorical_accuracy: 0.0664 - val_loss: 2.3414 - val_sparse_top_k_categorical_accuracy: 0.0626
Epoch 6/100
120/120 [

In [1]:
import pydot

keras.utils.plot_model(model, f"{outdir}/model.png", show_shapes=True)

NameError: name 'keras' is not defined

In [ ]:
S = tf.convert_to_tensor(np.random.randn(10,5,100,100,1))
Q = tf.convert_to_tensor(np.random.randn(10,15,100,100,1))

Y = model([Q, S])

Yp = tf.nn.softmax(Y, axis=-1)

In [ ]:
n_way = 10
q_shot = 15

scce = tf.keras.losses.SparseCategoricalCrossentropy()

Yt = np.tile(np.arange(n_way), (q_shot,1)).T.flatten()

scce(Yt, Yp)

In [ ]:
n_way = 10
q_shot = 15

scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Yt = np.tile(np.arange(n_way), (q_shot,1)).T.flatten()

scce(Yt, Y)

In [ ]:
Yt = np.tile(np.arange(n_way)[:,None], (1,q_shot))  # target labels

Yt = tf.reshape(tf.one_hot(Yt, depth=n_way, axis=-1), (-1,n_way))

# cce = np.mean(keras.losses.categorical_crossentropy(Yt, Yp, from_logits=False))

cce = tf.keras.losses.CategoricalCrossentropy()
cce(Yt, Yp)

In [ ]:
acc = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)
acc.update_state(Yt, Yp)
acc.result().numpy()

## EOF